In [1]:
import pdfplumber
import camelot.io as camelot
import tabula
from docx import Document
import pandas as pd
import pythoncom
from win32com.client import Dispatch
import requests

In [2]:
# Пример отчёта
example = """Основным видом деятельности предприятия СПК  «Рассвет» в соответствии с кодом ОКВЭД является выращивание зерновых и зернобобовых культур. 
Площадка № 1 (Производственная территория)
На территории производственной территории предприятия расположены следующие участки:
открытая стоянка автотранспорта;
нефтебаза;
крытый ток;
склад № 7.
Открытая стоянка автотранспорта
На балансе предприятия состоит 9 автомобилей. Хранение автотранспорта осуществляется на открытой стоянке на территории площадки предприятия. Одновременно с территории предприятия въезжает/выезжает не более одного автомобиля. В процессе прогрева двигателей и передвижения автотранспорта по территории предприятия непосредственно в атмосферу происходит неорганизованный выброс продуктов сгорания топлива в двигателях внутреннего сгорания автомобилей: азота диоксид, азота оксид, сажи, серы диоксида, углерод оксида, бензина, керосина (ИЗА №6001).
На балансе предприятия состоит 23 единицы дизельной сельскохозяйственной техники. Хранение сельскохозяйственной техники осуществляется на открытой площадке на территории предприятия. Одновременно с территории предприятия въезжает/выезжает не более одной единицы сельскохозяйственной техники. В процессе прогрева двигателей и передвижения техники по территории предприятия непосредственно в атмосферу происходит неорганизованный выброс продуктов сгорания дизельного топлива в двигателях сельскохозяйственной техники: азота диоксид, азота оксид, сажи, серы диоксида, углерод оксида, керосина (ИЗА №6002).

Нефтебаза
На территории производственной территории СПК «Рассвет» расположена собственная автозаправочная станция, на которой осуществляется заправка автотранспорта и техники предприятия. Склад ГСМ представляет собой наземные емкости, в которых осуществляется хранение дизельного топлива, бензина АИ-80 и масла МГ-26 для двигателей. Объем емкости с дизельным топливом составляет 25 м3, с бензином — 10 м3, с маслом — 5 м3. Закачка топлива в емкости происходит из бензовоза (с помощью насосов бензовоза). В процессе хранения и закачки топлива в резервуары в результате вытеснения паровоздушной смеси через дыхательные клапаны непосредственно в атмосферу происходит неорганизованный выброс следующих загрязняющих веществ: сероводород, смесь предельных углеводородов C1-C5, смесь углеводородов предельных C6-C10, пентилены, бензол, ксилол, толуол, этилбензол, алканы С12-С19 (ИЗА № 6003).
Заправка топлива в топливные баки и масла в двигатели сельскохозяйственной техники и автомобилей осуществляется с помощью трех топливораздаточных колонок «Нара», каждая из которых способна раздавать масло, бензин и дизельное топливо. Максимальный расход топлива приходится на весенне-летний период. В результате заправки топлива в баки сельскохозяйственной техники и автотранспорта, а также масла в двигатели непосредственно в атмосферу происходит неорганизованный выброс следующих загрязняющих веществ: сероводород, смесь предельных углеводородов C1-C5, смесь углеводородов предельных C6-C10, пентилены, бензол, ксилол, толуол, этилбензол, алканы С12-С19 (ИЗА № 6004).

Крытый ток
Сельскохозяйственные культуры (зерновые, подсолнечник) на территорию предприятия доставляется транспортом предприятия с сельскохозяйственных полей. В процессе разгрузки в результате работы двигателей автотранспорта в помещение крытого тока происходит выделение продуктов сгорания топлива: азота диоксида, азота оксида, сажи, серы диоксида, углерода оксида, бензина, керосина. Разгрузка самосвала происходит в помещении крытого тока, открытого с одной стороны. Разгрузка зерновых и подсолнечника не происходит одновременно. В результате разгрузки самосвала с сельскохозяйственными культурами в помещение крытого тока происходит выделение следующих веществ: пыль хлопковая, пыль зерновая. В летне-осенний период в крытом току осуществляется хранение сельскохозяйственных культур. В процессе хранения сельскохозяйственных культур в помещение крытого тока происходит выделение следующих загрязняющих веществ: пыли зерновой, пыли хлопковой. сельскохозяйственных культур. Погрузка сельскохозяйственных культур в транспорт заказчика осуществляется с помощью электрического зернометателя в помещении крытого тока. В процессе погрузки сельскохозяйственных культур зернометателем в транспорт заказчика происходит выделение следующих загрязняющих веществ: пыли зерновой, пыли хлопковой. В процессе погрузки в результате работы двигателей автотранспорта в помещение крытого тока происходит выделение продуктов сгорания топлива: азота диоксида, азота оксида, сажи, серы диоксида, углерода оксида, бензина, керосина. В результате вышеуказанных технологических процессов непосредственно в атмосферу через открытую часть крытого тока непосредственно в атмосферу происходит неорганизованный выброс следующих загрязняющих веществ: азота диоксида, азота оксида, сажи, серы диоксид, углерода оксида, бензина, керосина, пыли зерновой, пыли хлопковой (ИЗА №6005).

Склад № 7
Сельскохозяйственные культуры (зерновые, подсолнечник) на территорию предприятия доставляется транспортом предприятия с сельскохозяйственных полей. В процессе разгрузки в результате работы двигателей автотранспорта в помещение склада №7 происходит выделение продуктов сгорания топлива: азота диоксида, азота оксида, сажи, серы диоксида, углерода оксида, бензина, керосина. Разгрузка самосвала происходит в помещении закрытого склада № 7. Разгрузка зерновых и подсолнечника не происходит одновременно. В результате разгрузки самосвала с сельскохозяйственными культурами в помещение склада №7 происходит выделение следующих веществ: пыль хлопковая, пыль зерновая. В летне-осенний период на складе № 7 осуществляется хранение сельскохозяйственных культур. В процессе хранения сельскохозяйственных культур в помещение крытого тока происходит выделение следующих загрязняющих веществ: пыли зерновой, пыли хлопковой. Погрузка сельскохозяйственных культур в транспорт заказчика осуществляется с помощью электрического зернометателя в помещении склада №7. В процессе погрузки сельскохозяйственных культур зернометателем в транспорт заказчика происходит выделение следующих загрязняющих веществ: пыли зерновой, пыли хлопковой. В процессе погрузки в результате работы двигателей автотранспорта в помещение склада №7 происходит выделение продуктов сгорания топлива: азота диоксида, азота оксида, сажи, серы диоксида, углерода оксида, бензина, керосина. В результате вышеуказанных технологических процессов непосредственно в атмосферу через открытую часть крытого тока непосредственно в атмосферу происходит неорганизованный выброс следующих загрязняющих веществ: азота диоксида, азота оксида, сажи, серы диоксид, углерода оксида, бензина, керосина, пыли зерновой, пыли хлопковой (ИЗА №6006).
Отопление административного здания, расположенного на территории площадки №1 СПК «Рассвет» производится электрокотлом «ЭВПМ-9». Источники выброса загрязняющих веществ в процессе отопления отсутствуют.
Площадка № 2 (сельскохозяйственное поле)
На территории сельскохозяйственого поля расположены следующие участки:
Участок работы сельскохозяйственной техники;
Участок загрузки автомобиля.


Участок работы сельскохозяйственной техники
Для обработки поля привлекаются комбайны и трактора. В процессе работы сельскохозяйственной техники под нагрузкой происходит выделение продуктов сгорания топлива в двигателях сельскохозяйственной техники: азота диоксида, азота оксида, сажи, серы диоксид, углерода оксида, керосина. В процессе пересыпки сельскохозяйственных культур комбайном в автомобиль происходит выделение пыли хлопковой и пыли зерновой. В процессе работы двигателей сельскохозяйственной техники под нагрузкой и пересыпкой зерна в автомобиль непосредственно в атмосферу происходит неорганизованный выброс следующих загрязняющих веществ: азота диоксида, азота оксида, сажи, серы диоксид, углерода оксида, керосина, пыли хлопковой, пыли зерновой (ИЗА № 6007). В результате загрузки комбайном автомобилей предприятия непосредственно в атмосферу происходит неорганизованный выброс следующих загрязняющих веществ: азота диоксида, азота оксида, сажи, серы диоксид, углерода оксида, бензина (ИЗА №6008).
Информация о транспортных средствах и сельскохозяйственной техники, осуществляющих деятельность на территории предприятия СПК «Рассвет» представлена в Приложении 4. Справка о техническом обслуживании и ремонте транспортных средств и оборудования представлен в Приложении 4.
Техническое состояние оборудования СПК «Рассвет» — хорошее, эксплуатируется непродолжительное время, своевременно устраняются возникшие неполадки, регулярно производятся профилактические работы.
"""

pdf_path = 'Книга 1 Инвентаризация Био Агро Дон.pdf'

In [3]:
def extract_pdf_with_structure(pdf_path):
    # Преобразует pdf-файл в текст
    summaries = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            filtered = page_text.replace('-\n', '').replace('\n', ' ')
            summaries.append(f"Текст на странице {page_num}:\n{filtered}\n")

            tables = camelot.read_pdf(pdf_path, pages=str(page_num))
            for i, table in enumerate(tables):
                if table.parsing_report['accuracy'] > 10.: # фильтруем пустые таблицы
                    df = table.df
                    text = df.to_string().replace('-\\n', '').replace('\n', '')
                    title = f"Таблица {page_num}.{i + 1}" 
                    summaries.append(text)

    return summaries

In [4]:
def call_api_32k(system_prompt: str, prompt: str):
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion" 
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Api-Key AQVNwy5Z1_keoKl1F1R6BxAXLOKCiq5h1NEzd4Dw", # апи ключ
        "x-folder-id": "b1ghupvijk4nvsbqtcv3" # id каталога
    }
    
    payload = {
        "modelUri": "gpt://b1ghupvijk4nvsbqtcv3/yandexgpt-32k/rc", # модель https://yandex.cloud/ru/docs/foundation-models/concepts/yandexgpt/models

        # опции
        "completionOptions": {
            "stream": False,
            "temperature": 0.1,
            "maxTokens": "8192"
        },
        
        "messages": [
            {
                "role": "system",
                "text": system_prompt
            },
            {
                "role": "user",
                "text": prompt
            }
        ]
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        result = response.json()
        return result['result']['alternatives'][0]['message']['text']
    else:
        raise Exception(f"API request failed with status code {response.status_code}: {response.text}")

In [5]:
def summarize(text):
    # Суммаризует текст с помощью LLM
    # В системном промпте присутствует пример отчёта (One-Shot Learning)
    if len(text) > 100000:
        text = text[:100000] # ограничение YaGPT
    
    system_prompt = (
        "Вы — аналитическая система, создающая отчёты на основе данных. "
        "Вам предоставляется текст, включающий табличные данные. "
        "На основе этой информации необходимо создать подробный и логически структурированный отчёт, "
        "включая ключевые выводы и наблюдения. Пример отчёта:" + example
    )
    user_prompt = (
        f"Данные для анализа:\n{text}\n\n"
        "Пожалуйста, создайте отчёт на основе данных."
    )

    summarized = call_api_32k(system_prompt, user_prompt) # вызов LLM по API
    return summarized

In [6]:
summaries = extract_pdf_with_structure(pdf_path)
summaries

['Текст на странице 1:\nСправка о количестве источников выбросов во время их работы ООО “Био Агро Дон” № ИЗА Наименование № ИВ | Наименование | Время работы источника источника источника загрязнения выделения выделения, Ч атмосферы загрязняющих | вдень | в год веществ 1 2 3 4 5 6 6001 Неорганизованный | 01 | PaGora ДВС 1 260 спецтехники 02 Работа ДВС 1 260 автотранспорта Генеральный директор А. Н. Паршин 000 “Био Arpo Дон” ИИ аер\n',
 '        0                                                   1   2                                                                              3                                          4    50  №  ИЗА  Наименование \\nисточника \\nзагрязнения \\nатмосферы      №  ИВ  |  Наименование  | \\nисточника \\nвыделения \\nзагрязняющих  | \\nвеществ  Время  работы \\nисточника \\nвыделения,  Ч     1                                                                                                                                                                      

In [9]:
text = '\n'.join(summaries)
summarized = summarize(text)
summarized

'На основе предоставленных данных можно сделать следующие выводы:\n\n1. **Источники выбросов загрязняющих веществ:**\n- Неорганизованный выброс загрязняющих веществ происходит от дорожно-строительных машин, сельскохозяйственной техники и автотранспорта.\n- Выбросы включают такие вещества, как азота диоксид, азота оксид, сажа, серы диоксид, углерод оксид, бензин, керосин и другие.\n\n2. **Расчёт выбросов:**\n- Расчёты выбросов выполнены в соответствии с методическими документами, включая «Методическое пособие по расчёту, нормированию и контролю выбросов загрязняющих веществ в атмосферный воздух» и «Методика проведения инвентаризации выбросов загрязняющих веществ в атмосферу для баз дорожной техники».\n\n3. **Удельные выбросы:**\n- Приведены удельные выбросы загрязняющих веществ при работе пускового двигателя, прогреве, пробеге и на холостом ходу для различных типов дорожно-строительных машин.\n\n4. **Валовые и максимально разовые выбросы:**\n- Рассчитаны валовые и максимально разовые вы

In [10]:
document = Document()
p = document.add_paragraph(summarized.replace('*', ''))
document.save('summary.docx')